In [1]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from dotenv import load_dotenv
import os
from pydantic import Field, BaseModel
from typing import Dict, List, Any
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass
from presentation_generator import Presentation_gen
from typing import Optional
from spire.doc import Document,FileFormat
from spire.doc.common import *
import requests
from PIL import Image
from io import BytesIO
import tempfile
import pandas as pd



In [17]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')
pse=os.getenv('pse')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()
presentation=Presentation_gen()

@dataclass
class Deps:
    deep_search_results:dict
    quick_search_results:list[str]
    # presentation_slides:dict



async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query, generates a report or a paper.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_results=res
    return str(res)

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
async def quick_research_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_results.append(res.data)
    return str(res.data)


def google_image_search(query:str):
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": google_api_key,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  


async def research_editor_tool(ctx: RunContext[Deps], query:str):
    """
    Use this tool to edit the deep search result to make it more accurate following the query's instructions.
    This tool can modify paragraphs, image_url or table. For image_url, you need to give the query to search for the image.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    @dataclass
    class edit_route:
        paragraph_number:Optional[int] = Field(default_factory=None, description='the number of the paragraph to edit, if the paragraph is not needed to be edited, return None')
        route: str = Field(description='the route to the content to edit, either paragraphs, image_url, edit_table or add_table')

 
   
    paper_dict={'title':ctx.deps.deep_search_results.get('title'),
                                'image_url':ctx.deps.deep_search_results.get('image_url') if ctx.deps.deep_search_results.get('image_url') else 'None',
                                'paragraphs_title':{num:paragraph.get('title') for num,paragraph in enumerate(ctx.deps.deep_search_results.get('paragraphs'))}, 
                                'table':ctx.deps.deep_search_results.get('table') if ctx.deps.deep_search_results.get('table') else 'None',
                                'references':ctx.deps.deep_search_results.get('references')}
    
    route_agent=Agent(llm,result_type=edit_route, system_prompt="you decide the route to the content to edit based on the query's instructions and the paper_dict, either paragraphs, image_url or table")
    route=await route_agent.run(f'query:{query}, paper_dict:{paper_dict}')
    contents=ctx.deps.deep_search_results
 
    class Table_row(BaseModel):
        data: List[str] = Field(description='the data of the row')
    class Table(BaseModel): 
        rows: List[Table_row] = Field(description='the rows of the table')
        columns: List[str] = Field(description='the columns of the table')
    @dataclass
    class Research_edits:
        edits:str = Field(description='the edits')
    editor_agent=Agent(llm,tools=[google_image_search],result_type=Research_edits, system_prompt="you are an editor, you are given a query, some content to edit, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions, return only the edited content, no comments")
    if route.data.route=='paragraphs':
        content=contents.get('paragraphs')[route.data.paragraph_number]['content']
        res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results['paragraphs'][route.data.paragraph_number]['content']=res.data.edits
    if route.data.route=='image_url':
        content=contents.get('image_url')
        res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results['image_url']=res.data.edits
    if route.data.route=='edit_table':
        content=contents.get('table')
        table_editor_agent=Agent(llm, result_type=Table, system_prompt="edit the table content based on the query's instructions, the research and the quick search results")
        table=await table_editor_agent.run(f'query:{query}, table:{content}, research:{ctx.deps.deep_search_results}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results['table']={'data':[row.data for row in table.data.rows], 'columns':table.data.columns}
    if route.data.route=='add_table':
        table_agent=Agent(llm, result_type=Table, system_prompt="generate a detailed table in a dictionary format based on the research and the query")
        table=await table_agent.run(f'query:{query}, research:{ctx.deps.deep_search_results}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
        ctx.deps.deep_search_results['table']={'data':[row.data for row in table.data.rows], 'columns':table.data.columns}
    
    return str(ctx.deps.deep_search_results)


# async def presentation_agent(
#     ctx: RunContext[Deps],
#     style: Optional[str] = "professional",
#     instruction: Optional[str] = 'None'
# ) -> dict:
#     """
#     Generate a presentation based on the deep search result.

#     Args:
#         style: Presentation style (default: "professional")
#         instruction: Additional instructions for presentation generation (default: None)

#     Returns:
#         dict: The presentation slides
#     """
#     search_content = (ctx.deps.edited_deep_search_result 
#                      if ctx.deps.edited_deep_search_result 
#                      else ctx.deps.deep_search_results[-1])
    
#     res = await presentation.chat(search_content, presentation_style=style, instruction=instruction)
#     ctx.deps.presentation_slides = res
#     return 'presentation generated'

# @dataclass
# class page_edit:
#     edits: str = Field(description='the edited page in html format')

# page_editor=Agent(llm, result_type=page_edit, system_prompt='you are a page editor, edit the mentionned page(s) based on the instructions.')
# async def page_editor_tool(ctx: RunContext[Deps], page_number: int, instructions: str):
#     """
#     This function is used to edit the mentionned page(s) based on the instructions.
#     Args:
#         page_number (int): The number of the page to edit
#         instructions (str): The instructions for the page edits
#     Returns:
#         str: The edited page
#     """
#     result=await page_editor.run(f'page:{ctx.deps.presentation_slides[f"page_{page_number}"]}, instructions:{instructions}')
#     ctx.deps.presentation_slides[f"page_{page_number}"]=result.data.edits
#     return f'page{page_number} edited'


In [3]:
    # @dataclass
    # class edit_route:
    #     page_number:int = Field(description='the page number to edit')
    #     paragraph_number:Optional[int] = Field(default_factory=None, description='the number of the paragraph to edit, if the paragraph is not needed to be edited, return None')
    #     route: str = Field(description='the route to the content to edit, either paragraphs, image_url, edit_table or add_table')

    # paper_dict={}
    # for i in range(len(ctx.deps.deep_search_results)):
    #     paper_dict[f'page_{i+1}']={'title':ctx.deps.deep_search_results[i].get('title'),
    #                                'image_url':ctx.deps.deep_search_results[i].get('image_url') if ctx.deps.deep_search_results[i].get('image_url') else 'None',
    #                                'paragraphs_title':{num:paragraph.get('title') for num,paragraph in enumerate(ctx.deps.deep_search_results[i].get('paragraphs'))}, 
    #                                'table':ctx.deps.deep_search_results[i].get('table') if ctx.deps.deep_search_results[i].get('table') else 'None',
    #                                'references':ctx.deps.deep_search_results[i].get('references')}
    
    # route_agent=Agent(llm,result_type=edit_route, system_prompt="you decide the route to the content to edit based on the query's instructions and the paper_dict, either paragraphs, image_url or table")
    # route=await route_agent.run(f'query:{query}, paper_dict:{paper_dict}')
    # contents=ctx.deps.deep_search_results[route.data.page_number-1]
 
    
    # @dataclass
    # class Research_edits:
    #     edits:str = Field(description='the edits')
    # editor_agent=Agent(llm,tools=[google_image_search],result_type=Research_edits, system_prompt="you are an editor, you are given a query, some content to edit, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions, return only the edited content, no comments")
    # if route.data.route=='paragraphs':
    #     content=contents.get('paragraphs')[route.data.paragraph_number]['content']
    #     res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
    #     ctx.deps.deep_search_results[route.data.page_number-1]['paragraphs'][route.data.paragraph_number]['content']=res.data.edits
    # if route.data.route=='image_url':
    #     content=contents.get('image_url')
    #     res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
    #     ctx.deps.deep_search_results[route.data.page_number-1]['image_url']=res.data.edits
    # if route.data.route=='edit_table':
    #     content=contents.get('table')
    #     res=await editor_agent.run(f'query:{query}, content:{content}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}')
    #     ctx.deps.deep_search_results[route.data.page_number-1]['table']=res.data.edits
    # if route.data.route=='add_table':
 
    #     res=await editor_agent.run(f'query:{query}, quick_search_results:{ctx.deps.quick_search_results if ctx.deps.quick_search_results else "None"}, research_results:{ctx.deps.deep_search_results[route.data.page_number-1]}')
    #     ctx.deps.deep_search_results[route.data.page_number-1]['table']=res.data.edits
    
    # return str(ctx.deps.deep_search_results)

In [18]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

In [19]:
class Main_agent:
    def __init__(self):
        self.agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, leverage what tool(s) to use but do not use the research editor agent unless the user asks for it,\
                          always show the output of the tools",
                          tools=[deep_research_agent,research_editor_tool,quick_research_agent])
        self.deps=Deps( deep_search_results={}, quick_search_results=[])
        self.memory=Message_state(messages=[])

    async def chat(self, query:str):
        result = await self.agent.run(query,deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data
    
    def reset(self):
        self.memory.messages=[]
        self.deps=Deps( deep_search_results={}, quick_search_results=[])


In [20]:
agent=Main_agent()

In [21]:
await agent.chat('do a deep research on the topic of snakes in miami and generate a table of the snake species and their venomousness')

"Here is the information you requested from the deep research agent:\n\nTitle: Snakes in Miami: Identification and Safety\n\nImage URL: https://www.floridamuseum.ufl.edu/wp-content/uploads/sites/116/2020/08/IMG_2389_S.jpg\n\nParagraphs:\n\nMiami Snakes: Miami-Dade County, Florida is home to both venomous and non-venomous snakes. The venomous snakes include the eastern diamondback rattlesnake, timber rattlesnake, dusky pygmy rattlesnake, coral snake, cottonmouth, and copperhead. However, venomous snakes are rarely seen. Non-venomous snakes include the Brahminy Blind Snake, a small, non-native species. Other snakes, like pythons, have become established in Florida, particularly in the Everglades. Resources like the Florida Museum of Natural History and Miami-Dade Fire Rescue offer guides for identifying snakes and dealing with bites and stings.\n\nCommon Snake Species in Miami: Miami-Dade County is home to a variety of snake species, both venomous and non-venomous. Common non-venomous sn

In [14]:
def temp_image(url:str):
    image=requests.get(url)
    image=Image.open(BytesIO(image.content))
    with tempfile.NamedTemporaryFile(mode='w', suffix='.png', delete=False) as temp_file:
        image.save(temp_file.name)
        image_file_path = temp_file.name
    return image_file_path

In [15]:
def paper_to_markdown(paper:dict) -> str:
    """
    Convert a Paper_layout object into a markdown string.
    
    Args:
        paper (Paper_layout): The paper layout to convert
        
    Returns:
        str: The complete markdown document
    """
    
    markdown = []
    
    # Add title
    markdown.append(paper.get('title'))
    markdown.append("\n\n")
    if paper.get('image_url'):
        markdown.append(f"![Image]({paper.get('image_url')})\n")
        markdown.append("\n\n")
    
    # Add body sections
    for section in paper.get('paragraphs'):
        markdown.append(f"## {section.get('title')}\n")
        if section.get('content'):
            markdown.append(section.get('content'))
            markdown.append("\n")


        markdown.append("\n\n")
    
    if paper.get('table'):
        markdown.append(pd.DataFrame(paper.get('table')).to_markdown())
        markdown.append("\n")
    # Add references
    markdown.append("## References\n")
    markdown.append(str(paper.get('references')))
    
    return "".join(markdown)

In [16]:
# paper=paper_to_markdown(dict)

In [17]:
# print(paper)

In [18]:



# Create a temporary file
# with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as temp_file:
#     temp_file.write(paper)
#     temp_file_path = temp_file.name

# # Create a new document and load from the temporary files
# document = Document()
# document.LoadFromFile(temp_file_path)

# # Save as DOCX
# document.SaveToFile("output.docx", FileFormat.Docx)

# # Clean up the temporary file
# os.unlink(temp_file_path)

In [49]:
data={'title': '# Snakes in Miami: A Comprehensive Overview',
 'image_url': 'https://www.evergladesholidaypark.com/wp-content/uploads/coral-snake.jpg',
 'paragraphs': [{'title': 'Introduction',
   'content': "Florida is home to a diverse snake population, including both native and non-native species. The Florida Fish and Wildlife Conservation Commission (FWC) plays a crucial role in managing these species, enacting regulations to protect the public and the state's ecosystems. Recent rule changes have added high-risk nonnative reptiles to Florida's Prohibited list, limiting possession to specific purposes. South Florida is home to a large number of snake species, including both native and non-native, with the Burmese python being a major concern due to its devastating impact on native wildlife in the Everglades. Only six of Florida's 44 snake species are venomous, and resources are available to help identify both venomous and non-venomous snakes, as well as to provide guidance on how to handle encounters safely. Snake removal services are available in areas like Miami-Dade County to address snake-related issues and ensure public safety."},
  {'title': 'Native Snake Species in Miami',
   'content': "South Florida is home to a diverse snake population, including both native and non-native species. Of Florida's 46 native snake species, 34 are found in South Florida, including four of the six venomous species found in the state. The Florida Fish and Wildlife Conservation Commission (FWC) manages snake-related regulations, including those pertaining to venomous reptiles and invasive species like the Burmese python, which poses a significant threat to native wildlife in the Everglades. The FWC also provides resources for identifying snakes and promoting safety. Miami-Dade County residents can seek assistance from specialized wildlife removal services for snake control and removal, particularly for venomous snakes or those found in residential areas."},
  {'title': 'Non-Native Snake Species in Miami',
   'content': "South Florida is home to a diverse snake population, including both native and non-native species. The Florida Fish and Wildlife Conservation Commission (FWC) regulates snake ownership and has implemented rules to manage high-risk nonnative reptiles, such as adding 16 species to Florida's Prohibited list. Burmese pythons are a major concern as an invasive species in the Everglades, preying on native wildlife and causing ecological shifts. Miami-Dade County offers snake removal services for both venomous and non-venomous snakes, with experts specializing in wild animal removal."},
  {'title': 'Venomous Snakes of Miami',
   'content': "South Florida is home to a diverse snake population, including both native and non-native species, with about 47 species in total. Among Florida's 44 native snake species, 34 are found in South Florida, with four of them being venomous. The presence of venomous snakes like the Eastern Diamondback Rattlesnake, Cottonmouth, and Coral Snake necessitates caution and awareness. However, non-native species like the Burmese python pose a significant threat to the Everglades ecosystem, preying on native wildlife and causing severe declines in mammal populations. The Florida Fish and Wildlife Conservation Commission (FWC) regulates snake ownership and has implemented rules to manage high-risk nonnative reptiles, including restrictions on possession and efforts to control or eradicate certain species. Residents are encouraged to learn how to identify snakes and coexist safely with them, utilizing resources like updated guides with safety recommendations and descriptions of both venomous and non-venomous species."},
  {'title': 'The Impact of Burmese Pythons on the Everglades Ecosystem',
   'content': 'Burmese pythons, an invasive species in South Florida, particularly the Everglades, pose a significant threat to the native ecosystem. They compete with native wildlife for food, leading to severe declines in mammal populations and preying on species like bobcats. The Florida Fish and Wildlife Conservation Commission (FWC) has implemented regulations, including adding high-risk nonnative reptiles to the Prohibited list, to manage snake ownership and prevent the escape of venomous reptiles. While South Florida is home to around 47 snake species, including both native and non-native ones, increased awareness and resources are available to help residents distinguish between venomous and non-venomous snakes and address snake-related issues through specialized removal services.'},
  {'title': 'Snake Removal and Control in Miami-Dade County',
   'content': "Miami-Dade County faces challenges with both native and non-native snake species. South Florida is home to 34 of Florida's 46 native snake species, including some venomous ones, requiring residents to be cautious and aware. The introduction of non-native species, particularly the Burmese python, has significantly impacted the Everglades ecosystem, leading to declines in native mammal populations and posing a threat to wildlife like bobcats. The Florida Fish and Wildlife Conservation Commission (FWC) regulates snake ownership and has implemented rules to manage high-risk nonnative reptiles, focusing on control and eradication efforts. Residents seeking snake removal and control services in Miami-Dade County should seek out specialized wildlife removal professionals rather than general pest control companies, ensuring safe and effective handling of both venomous and non-venomous snakes."},
  {'title': 'Regulations and Conservation Efforts',
   'content': "Florida's approach to snake management involves a combination of regulations, conservation efforts, and public education. The Florida Fish and Wildlife Conservation Commission (FWC) plays a central role, enacting and enforcing rules to manage both native and non-native snake populations. Regulations include prohibitions on certain high-risk nonnative reptiles, restrictions on possession, and guidelines for snake ownership, focusing on public safety and ecosystem protection. Conservation efforts target threatened species like the Salt Marsh Water Snake, while also addressing the impact of invasive species such as the Burmese python through research and removal programs. Public education initiatives, including updated identification guides, aim to help residents differentiate between venomous and non-venomous species and promote safe coexistence with Florida's diverse snake population."},
  {'title': 'Conclusion',
   'content': "Florida is home to a diverse snake population, including 44 native species, six of which are venomous, and several non-native species. South Florida alone hosts around 47 snake species, both native and non-native. The Florida Fish and Wildlife Conservation Commission (FWC) manages these snake populations through regulations, including rules about owning certain species and preventing the escape of venomous reptiles. A significant concern is the invasive Burmese python, which has established a breeding population in South Florida and is causing severe declines in native mammal populations in the Everglades. The FWC has added 16 high-risk nonnative reptiles to Florida's Prohibited list. Efforts are underway by various organizations to control and remove these pythons to mitigate their impact on the ecosystem. Resources such as updated identification guides and snake removal services are available to help residents coexist safely with Florida's snake population."}],
 'table': 'Here is a table showing some snake species and whether they are venomous, based on the provided content.\n\n| Snake Species | Venomous? |\n|---|---| \n| Eastern Diamondback Rattlesnake | Yes |\n| Cottonmouth | Yes |\n| Coral Snake | Yes |\n| Burmese python | No |\n\nReferences:\nhttps://www.fishwildlife.org/law-research-library/law-categories/reptiles-amphibians/reptiles-amphibians-enforcement-and-penalties-florida\nhttps://myfwc.com/wildlifehabitats/nonnatives/regulations/snakes-and-lizards/\nhttps://myfwc.com/about/rules-regulations/\nhttps://legalclarity.org/florida-snake-ownership-laws-and-compliance-guide/\nhttps://myfwc.com/conservation/you-conserve/wildlife/snakes/',
 'references': 'https://www.fishwildlife.org/law-research-library/law-categories/reptiles-amphibians/reptiles-amphibians-enforcement-and-penalties-florida, https://myfwc.com/wildlifehabitats/nonnatives/regulations/snakes-and-lizards/, https://myfwc.com/about/rules-regulations/, https://legalclarity.org/florida-snake-ownership-laws-and-compliance-guide/, https://myfwc.com/conservation/you-conserve/wildlife/snakes/'}

In [50]:
query='generate a table of the snake species and their venomousness'

In [118]:
class Table_row(BaseModel):
    data: List[str] = Field(description='the data of the row')
class Table(BaseModel):
    rows: List[Table_row] = Field(description='the rows of the table')
    columns: List[str] = Field(description='the columns of the table')
    

In [119]:
table_agent=Agent(llm, result_type=Table, system_prompt="generate a detailed table in a dictionary format based on the research and the query")
table=await table_agent.run(f'query:{query}, research:{data}')


In [120]:
df=pd.DataFrame(data=[row.data for row in table.data.rows], columns=table.data.columns)

In [123]:
dickt=pd.DataFrame.to_dict(df)

In [125]:
pd.DataFrame(dickt)

,Snake Species,Venomous?
0,Eastern Diamondback Rattlesnake,Yes
1,Cottonmouth,Yes
2,Coral Snake,Yes
3,Burmese python,No
